##### **1. Setup File, Paths, & Libraries**

In [ ]:
import os 
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

DATA_PATH = r"C:\Users\Fatma\OneDrive-Work\OneDrive\Solutions_by_STC\Tasks\Week_2\HR_Project\data\WA_Fn-UseC_-HR-Employee-Attrition.csv"  
CLEANED_CSV = r"data\cleaned_hr.csv"
DB_PATH = r"employees.db"

print("DATA_PATH:", DATA_PATH)
print("CLEANED_CSV:", CLEANED_CSV)
print("DB_PATH:", DB_PATH)



DATA_PATH: C:\Users\Fatma\OneDrive-Work\OneDrive\Solutions_by_STC\Tasks\Week_2\HR_Project\data\WA_Fn-UseC_-HR-Employee-Attrition.csv
CLEANED_CSV: data\cleaned_hr.csv
DB_PATH: employees.db


In [2]:
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"CSV not found at: {DATA_PATH}")
else:
    print("CSV found at:",DATA_PATH, "and ready to load")
    
    

CSV found at: C:\Users\Fatma\OneDrive-Work\OneDrive\Solutions_by_STC\Tasks\Week_2\HR_Project\data\WA_Fn-UseC_-HR-Employee-Attrition.csv and ready to load


##### **2. Loaded The Dataset**

In [ ]:
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Loaded CSV with {df.shape[0]} rows and {df.shape[1]} columns")

except Exception as e:
    raise RuntimeError("Failed to read CSV file with pandas:" +str(e))

print("\n--- Preview of Data ---")
print(df.head().to_string())
    

Loaded CSV. Rows,Cols: (1470, 35)

--- head() sample ---
   Age Attrition     BusinessTravel  DailyRate              Department  DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  EnvironmentSatisfaction  Gender  HourlyRate  JobInvolvement  JobLevel                JobRole  JobSatisfaction MaritalStatus  MonthlyIncome  MonthlyRate  NumCompaniesWorked Over18 OverTime  PercentSalaryHike  PerformanceRating  RelationshipSatisfaction  StandardHours  StockOptionLevel  TotalWorkingYears  TrainingTimesLastYear  WorkLifeBalance  YearsAtCompany  YearsInCurrentRole  YearsSinceLastPromotion  YearsWithCurrManager
0   41       Yes      Travel_Rarely       1102                   Sales                 1          2  Life Sciences              1               1                        2  Female          94               3         2        Sales Executive                4        Single           5993        19479                   8      Y      Yes                 11                

In [ ]:
print("\n--- Dataset Information ---")
df.info()

print("\n--- Missing Values for Each Column ---")
print(df.isnull().sum())

print("\n--- Total Duplicate Rows ---")
print(df.duplicated().sum())



--- info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel  

##### **3. Cleaned The Data**

In [ ]:
df = df.drop_duplicates()
str_cols = df.select_dtypes(include="object").columns.tolist()
for c in str_cols:
    df[c] = df[c].astype(str).str.strip()



no_cols = ['EmployeeNumber', 'Age', 'DailyRate', 'DistanceFromHome', 'HourlyRate', 'MonthlyIncome',
            'NumCompaniesWorked', 'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear',
            'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'PerformanceRating']

for c in no_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print("\nCount of Missing Values in Each Column:")
print(df[no_cols].isnull().sum())


Nulls after numeric coercion (per column):
EmployeeNumber             0
Age                        0
DailyRate                  0
DistanceFromHome           0
HourlyRate                 0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsInCurrentRole         0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
PerformanceRating          0
dtype: int64

Cleaned Shape: (1470, 35)


##### **4. Handled The Missing Values**

In [ ]:
if 'MonthlyIncome' in df.columns and df['MonthlyIncome'].isnull().any():
    print('\nDropping The Rows with Missing MonthlyIncome - count:', df['MonthlyIncome'].isnull().sum())
    df = df.dropna(subset=['MonthlyIncome'])


if 'PerformanceRating' in df.columns and df['PerformanceRating'].isnull().any():
    med = df['PerformanceRating'].median()
    print(f"Replacing missing PerformanceRating with median value: {med}")
    df['PerformanceRating'] = df['PerformanceRating'].fillna(med)

print("\nCleaned Shape:", df.shape)

##### **5. Saved The Cleaned Data**

In [ ]:
os.makedirs(os.path.dirname(CLEANED_CSV), exist_ok=True)
df.to_csv(CLEANED_CSV, index=False)
print('Saved The Cleaned CSV to', CLEANED_CSV)

Saved Cleaned CSV to data\cleaned_hr.csv


In [ ]:
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)
df.to_sql('employees', engine, if_exists='replace', index=False)
print("Wrote The Table of 'employees' to SQLite DB at", DB_PATH)


sample = pd.read_sql_query('SELECT COUNT(*) AS total_employees FROM employees', engine)
print('\nThe Total Number of Employees in The Database (via SQL) is:')
print(sample)

Wrote table 'employees' to SQLite DB at employees.db

Total employees in DB (via SQL):
   total_employees
0             1470


##### **6. Finally Analyzed HR DATA**

In [18]:
#Q1) How many total employees are there?
q1 = "SELECT COUNT(*) AS total_employees FROM employees;"
print('\nAns 1 - Total Number of Employees is:')
print(pd.read_sql_query(q1, engine))


#Q2) What is the employee count for each department?
q2 = 'SELECT Department, COUNT(*) AS cnt FROM employees GROUP BY Department ORDER BY cnt DESC;'
print('\nAns 2 - The Employee Count for Each Department:')
print(pd.read_sql_query(q2, engine))

      
#Q3) What is the average monthly income by job role?
q3 = 'SELECT JobRole, ROUND(AVG(MonthlyIncome),2) AS avg_monthly_income FROM employees GROUP BY JobRole ORDER BY avg_monthly_income DESC;'
print('\nAns 3 - The Average Monthly Income by Job Role:')
print(pd.read_sql_query(q3,engine))


#Q4) Who are the top 5 employees by performance rating?
q4 = 'SELECT EmployeeNumber, Age, JobRole, PerformanceRating, MonthlyIncome FROM employees ORDER BY PerformanceRating DESC, MonthlyIncome DESC LIMIT 5;'
print('\nAns 4 - The Top 5 Employees by Performance Rating:')
print(pd.read_sql_query(q4, engine))


#Q5) Which department has the highest average performance rating?
q5 = 'SELECT Department, ROUND(AVG(PerformanceRating),2) AS avg_perf FROM employees GROUP BY Department ORDER BY avg_perf DESC LIMIT 1;'
print('\nAns 5 - Department with The Highest Average Performance Rating:')
print(pd.read_sql_query(q5, engine))

#Q6) Which department has the highest attrition rates?
q6 = """SELECT Department, ROUND(SUM(CASE WHEN Attrition="Yes" THEN 1 ELSE 0 END)*100.0/COUNT(*),2) AS attrition_rate
    FROM employees GROUP BY Department ORDER BY attrition_rate DESC LIMIT 1;"""
print("\n Ans 6 - Department with The Highest Attrition Rates:")
print(pd.read_sql_query(q6, engine))

#Q7) Does working overtime affect job satisfaction, attrition, or performance?
q7 = """SELECT OverTime, ROUND(AVG(JobSatisfaction),2) AS avg_job_satisfaction,
ROUND(AVG(PerformanceRating),2) AS avg_performance,
ROUND(SUM(CASE WHEN Attrition='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*),2) AS attrition_rate 
FROM employees GROUP BY OverTime ORDER BY attrition_rate DESC;"""
print("\nAns 7 - Impact of OverTime on Job Satisfaction, Performance, and Attrition:")
print(pd.read_sql_query(q7, engine))

#Q8) Does living farther from work increase attrition or decrease job satisfaction?
q8 = """SELECT CASE WHEN DistanceFromHome <= 5 THEN 'O-5km'
    WHEN DistanceFromHome <= 10 THEN '6-10km'
    WHEN DistanceFromHome <= 15 THEN '11-15km'
    ELSE '20+ km' END AS distance_group, ROUND(AVG(JobSatisfaction),2) AS avg_job_satisfaction,
    ROUND(SUM(CASE WHEN Attrition='Yes' THEN 1 ELSE 0 END)*100.0/COUNT(*),2) AS attrition_rate
    FROM employees GROUP BY distance_group ORDER BY distance_group;"""
print("\nAns 8 - Impact of Distance From Home on Job Satisfaction and Attrition:")
print(pd.read_sql_query(q8, engine))

      




Ans 1 - Total Number of Employees is:
   total_employees
0             1470

Ans 2 - The Employee Count for Each Department:
               Department  cnt
0  Research & Development  961
1                   Sales  446
2         Human Resources   63

Ans 3 - The Average Monthly Income by Job Role:
                     JobRole  avg_monthly_income
0                    Manager            17181.68
1          Research Director            16033.55
2  Healthcare Representative             7528.76
3     Manufacturing Director             7295.14
4            Sales Executive             6924.28
5            Human Resources             4235.75
6         Research Scientist             3239.97
7      Laboratory Technician             3237.17
8       Sales Representative             2626.00

Ans 4 - The Top 5 Employees by Performance Rating:
   EmployeeNumber  Age            JobRole  PerformanceRating  MonthlyIncome
0            1035   41  Research Director                  4          19973
1      